# Dog vs. Cat Classifier
This notebook is supposed to experiment on different aspects of a image classifier: types of layers (fully connected, convolutional), training sample sizes, image generator, pretrained models, etc. 

## Image Generator tools from _keras_

In [1]:
from keras.preprocessing.image import ImageDataGenerator
#from PIL import Image

Using TensorFlow backend.


Demo of how the data generator works: 

In [9]:
G = 2 # number of GPU's

Prepare the data, using the .flow_from_directory() method. 

In [24]:
batch_size = 64

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator( 
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
        rescale=1./255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/scratch/bnallamo_armis/wsliu/dogs_cats/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size*G,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/scratch/bnallamo_armis/wsliu/dogs_cats/data/train2000',
        target_size=(224, 224),
        batch_size=batch_size*G,
        class_mode='categorical')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Model specification

In [11]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
import tensorflow as tf
from keras.utils import multi_gpu_model

In [28]:
# build the model on CPU memory:
with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2,2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2,2)))

    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2,2)))

    model.add(Conv2D(256, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2,2)))
# the model so far outputs 3D feature maps (height, width, features)

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))

In [29]:
# duplicate model on each GPU
parallel_model = multi_gpu_model(model, gpus=G)

#sgd = SGD(lr=0.5, decay=1e-6, momentum=0.5, nesterov=True)
parallel_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model training:

In [30]:
parallel_model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // (batch_size*G),
        epochs=5,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // (batch_size*G));

Epoch 1/5
179/179 [==============================] - 325s 2s/step - loss: 0.6376 - acc: 0.6220 - val_loss: 0.5653 - val_acc: 0.7021
Epoch 2/5
179/179 [==============================] - 324s 2s/step - loss: 0.5516 - acc: 0.7153 - val_loss: 0.4878 - val_acc: 0.7693
Epoch 3/5
121/179 [===================>..........] - ETA: 1:40 - loss: 0.4959 - acc: 0.7572

KeyboardInterrupt: 

In [6]:
model.save_weights('naive_cnn_full_sample.h5')  # always save your weights after training or during training